In [2]:
%pip install matplotlib

   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   --------------------- ------------------ 4.5/8.1 MB 24.4 MB/s eta 0:00:01
   -------------------------------------- - 7.9/8.1 MB 28.6 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 20.1 MB/s  0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 2.3/2.3 MB 18.3 MB/s  0:00:00

   -------- ------------------------------- 1/5 [fonttools]
   -------- ------------------------------- 1/5 [fonttools]
   -------- ------------------------------- 1/5 [fonttools]
   -------- ------------------------------- 1/5 [fonttools]
   -------- ------------------------------- 1/5 [fonttools]
   -------- ------------------------------- 1/5 [fonttools]
   -------- ------------------------------- 1/5 [fonttools]
   -------- ------------------------------- 1/5 [fonttools]
   -------- ------------------------------- 1/5 [fonttools]
   -------- -------

In [3]:
# ==============================
# 02_EDA_Vendas.ipynb
# ==============================

from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# Configuração de estilo
plt.style.use("seaborn-v0_8-whitegrid")

# Caminhos
ROOT = Path(".").resolve()
CLEAN_PATH = ROOT / "data" / "processed" / "data_clean.csv"
FIG_DIR = ROOT / "reports" / "figures"
FIG_DIR.mkdir(parents=True, exist_ok=True)

# ---------------------------
# 1. Carregar dataset limpo
# ---------------------------
df = pd.read_csv(CLEAN_PATH, parse_dates=["Data"])
df["Quantidade"] = pd.to_numeric(df["Quantidade"], errors="coerce").fillna(0).astype(int)
df["Preco"] = pd.to_numeric(df["Preco"], errors="coerce").fillna(0).astype(float)

# Total de vendas
df["Total"] = (df["Quantidade"] * df["Preco"]).round(2)

print("Amostra do dataset limpo:")
display(df.head())

# ---------------------------
# 2. Gráfico de linha: tendência mensal
# ---------------------------
# Agrupar por mês-ano
df["AnoMes"] = df["Data"].dt.to_period("M").astype(str)
mensal = df.groupby("AnoMes")["Total"].sum().reset_index()

# Plot
plt.figure(figsize=(10,5))
plt.plot(mensal["AnoMes"], mensal["Total"], marker="o")
plt.title("Tendência de Vendas Mensais (R$)")
plt.xlabel("Mês")
plt.ylabel("Total de Vendas (R$)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(FIG_DIR / "tendencia_mensal.png", dpi=120)
plt.show()

# ---------------------------
# 3. Gráfico de barras: Top-5 produtos
# ---------------------------
vendas_produto = df.groupby("Produto")["Total"].sum().sort_values(ascending=False).head(5)

plt.figure(figsize=(8,5))
vendas_produto.plot(kind="bar", color="royalblue")
plt.title("Top-5 Produtos em Vendas Totais")
plt.ylabel("Total de Vendas (R$)")
plt.xlabel("Produto")
plt.tight_layout()
plt.savefig(FIG_DIR / "top5_produtos.png", dpi=120)
plt.show()

# ---------------------------
# 4. Insights iniciais
# ---------------------------
print("=== Insights Observados ===")
print("1) Há variação clara no volume de vendas entre os meses — possível sazonalidade.")
print("2) Poucos produtos concentram a maior parte da receita (efeito 'cauda longa').")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\DEV\\Teste_Analytics_CristovamPaulo\\Parte01\\data\\processed\\data_clean.csv'